In [1]:
def convert_dict(fname):#this function is to convert json dictionary into a python dictionary and do little bit of text-cleaning   
    doc=[]
    result_title=[]
    result_text=[]
    ans_text=[]
    ball=[]

    with open(fname, 'r') as fi:
        for i in fi:
            null=None#as there are null values in the json dictionary, so converting them as None
            doc.append(eval(i))#converting a json dict into a python dict. tried to do this conversion using ast.literal_eval which is much safer than eval since it will only evaluate strings which contain literals
            #but that posed an error because of "string" literal in the json dictionary
    for ele in doc:#get the values from the python dictionary, doc and appending the values to two separate lists 
        if "text" in ele:
            result_text.append(str(ele["text"]).lower().replace("\n", "").replace("/", ""))#getting values of the dictionary key: text and appending it to result_text
        if "title" in ele:
            result_title.append(str(ele["title"]).lower().replace("\n", "").replace("/", ""))#getting values of the dictionary key: title and appending it to result_title
    return result_text, result_title

def stop_lemmatize(rtext):#this function comprises preprocessing steps re cleaning the text: first tokenize after removing punctuation, second remove insignificant words that can be found in stopwords
    #third use WordNetLemmatizer to get common base form of words. I ended up doing all these steps manually as the CountVectorizer do not have lemmatization in play
    from nltk.tokenize import word_tokenize
    from nltk.stem.wordnet import WordNetLemmatizer
    from nltk.corpus import stopwords
    import string
    rtext = [''.join(c for c in s if c not in string.punctuation) for s in rtext]#removing punctuation
    docs = [word_tokenize(content) for content in rtext]#tokenizing the texts into words
    stop = set(stopwords.words('english'))
    wordnet = WordNetLemmatizer()
    doc_fair = [[word for word in words if word not in stop] for words in docs]#removing words that are in stopwords
    docs_w = [' '.join(wordnet.lemmatize(w1).replace("‘", '').replace("’", '').replace("'", '') for w1 in w2) for w2 in doc_fair]#lemmatizing to get a common base form of words. finally joining the words to get a neat list of strings to pass in to the CountVectorizer
    return docs_w

def final_lists(f_name):
    result_text, result_title = f_name
    docs_wordnet=stop_lemmatize(result_text)
    docs_wordnet_title=stop_lemmatize(result_title)
    return docs_wordnet, docs_wordnet_title

In [3]:
#assigning the outcome of the above function: final_lists to a dataframe columns for an easy access to get the titles to associate with the clusters to get an idea 
#on how the algorithm have grouped the texts
import pandas as pd
df1=pd.DataFrame()
df1['text_document'], df1['title']=final_lists(convert_dict("/Users/user_name/Downloads/corpus.txt"))
                                               

In [4]:
df1.head()

,text_document,title
0,joe raedlegetty imageslast week wrote story sl...,reason skeptical teslapowered puerto rico
1,zach boydenholmesthe registerpolice identified...,police threat west de moines student work copycat
2,chicago police investigating whether stephen p...,gunman reserved two room blackstone
3,nashville tenn 5 georgia coming two impressive...,5 georgia bulldog vanderbilt commodore preview...
4,,craig robinson adam scott buddy fox supernatur...


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features =15000

tf_v = CountVectorizer(max_df=0.95, min_df=1, max_features=n_features, lowercase=False)#lowercase=False as the texts were converted into lowercase in the above steps
tf_x = tf_v.fit_transform(df1['text_document'])#to get X_values


In [7]:
from sklearn.decomposition import LatentDirichletAllocation
n_components = 60
lda = LatentDirichletAllocation(n_components=n_components, doc_topic_prior=0.83, topic_word_prior=0.1, learning_method='online', learning_offset=50.,random_state=0).fit(tf_x)

In [8]:
def print_top_words(model, feature_names, n_top_words):#this function finds the top words(taken from sklearn)
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words -1:-1]])
        print(message)
    print()

In [9]:
#The top 20 words in 60 clusters using the entire dataset
n_top_words = 20
#print("\nTopics in LDA model:")
tf_feature_names = tf_v.get_feature_names()
# len(tf_feature_names)
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0: access digital content range article search wide every provides custom entire topic reader license suite continue sky alert patent allows
Topic #1: data service security system network information customer user platform software solution technology internet application use provider communication cloud computer using
Topic #2: percent bank price stock fund investor money financial market trading payment billion rate trade dollar rose bitcoin higher index investment
Topic #3: said police county fire people man officer death area found child two family resident road told official near killed victim
Topic #4: free local day 30 today trial miss digital unlimited storynsign accessnsubscribe said string say new year maine including one people
Topic #5: million per company share tax 2017 quarter statement cent result income net revenue cost increase sale rate loss property cash
Topic #6: march cut jan monday change string close tuesday said new week cover paid also one time year thre

In [10]:
#NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
n_top_words = 20
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2)
tfidf = tfidf_vectorizer.fit_transform(df1['text_document'])

In [11]:
print("\nTopics in NMF model (Frobenius norm):")
nmf = NMF(n_components=60, random_state=1,alpha=.1, l1_ratio=.5).fit(tfidf)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, n_top_words)


Topics in NMF model (Frobenius norm):
Topic #0: said one people year like time new would say get day also make think could know thing first want way
Topic #1: string subscribe world comment author download 2018 bloomberg everyonenthis everyonenthe everyone every everyones everyday everybodys everything everythings everybody everytown everyman
Topic #2: company service business technology customer solution product industry platform client network management information new global digital software system financial partner
Topic #3: trump president donald white house administration american obama secretary policy america campaign washington immigration tweet bannon news congress republican meeting
Topic #4: rate growth inflation economy fed price economist bond economic policy increase yield interest wage year monetary higher spending consumer central
Topic #5: police officer man said suspect arrested victim incident found shot investigation county charged pm block killed according told 

In [12]:
#To get the top 5 popular events in every cluster
import numpy as np
nmf_embedding = nmf.transform(tfidf)
nmf_embedding = (nmf_embedding - nmf_embedding.mean(axis=0))/nmf_embedding.std(axis=0)
top_idx = np.argsort(nmf_embedding,axis=0)[-5:]

In [13]:
#Associating the clusters with title in the dataframe to get an idea and appropriate texts
def title_text_clusters(g):
    c=0
    for idxs in g.T:
        print("\nTopic {}:".format(c,))
        print("Title:{}".format(df1.iloc[idxs]['title']))
        c += 1

title_text_clusters(top_idx)


Topic 0:
Title:5293                         string palin harbinger trump
6504    string woman forced eat picture dead relative ...
5353                string brian snitker qa   going good 
2041     comic sebastian maniscalco phoenix 114 dont wish
5981      string black atheist white santa feast deceased
Name: title, dtype: object

Topic 1:
Title:8202       string former employee michael douglas fondled
5537         string globenewswire bond market information
8314    string social medium sweep saquons cleat penn ...
8472    string lehigh men  basketball team go cold fal...
1497    string ousted catalan leader challenge eu spea...
Name: title, dtype: object

Topic 2:
Title:4688    string rony sawdayi join bridg chief technolog...
9334    string frogworks brings enterprise level secur...
8882    string nec announces organizational change aim...
3173    string avitus group significantly enhances ser...
5391         string moneypenny acquires made simple group
Name: title, dtype: object

